Some examples from Raschka with our Dataset 

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
def slice_the_set(df,start,end):

    start = int(len(df)*start/100)
    end = int(len(df)*end/100)

    return df.iloc[start:end]

In [3]:
df =pd.read_csv("..\data\export_dataframe.csv")
le = preprocessing.LabelEncoder()
le.fit([ "A", "D", "H"])
df.loc[:,['result']]=le.transform(df['result'])

In [4]:
df.head()

,result,odds-home,odds-draw,odds-away,home-wins,home-draws,home-losses,home-goals,home-opposition-goals,away-wins,away-draws,away-losses,away-goals,away-opposition-goals
0,0,1.36,4.75,8.50,7,1,2,26,12,3,6,1,11,6
1,1,2.40,3.30,2.90,2,3,5,9,14,5,2,3,17,12
2,1,3.20,3.30,2.25,4,2,4,11,13,7,2,1,19,8
3,2,1.75,3.75,4.33,4,1,5,14,16,4,1,5,13,12
4,1,1.40,4.33,8.00,5,3,2,21,11,2,3,5,14,20


In [5]:
y = df.iloc[:,0]
y.head()

0    0
1    1
2    1
3    2
4    1
Name: result, dtype: int32

In [6]:
X = df.iloc[:,1::]
X.head()

,odds-home,odds-draw,odds-away,home-wins,home-draws,home-losses,home-goals,home-opposition-goals,away-wins,away-draws,away-losses,away-goals,away-opposition-goals
0,1.36,4.75,8.50,7,1,2,26,12,3,6,1,11,6
1,2.40,3.30,2.90,2,3,5,9,14,5,2,3,17,12
2,3.20,3.30,2.25,4,2,4,11,13,7,2,1,19,8
3,1.75,3.75,4.33,4,1,5,14,16,4,1,5,13,12
4,1.40,4.33,8.00,5,3,2,21,11,2,3,5,14,20


In [7]:
#from sklearn.cross_validation import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#TODO: Delete the following statements and import sklearn.cross_validation

df_train = slice_the_set(df,0,80)
df_test = slice_the_set(df,80,100)

y_train  = df_train['result']
X_train= df_train.iloc[:,1:-1]
y_test = df_test['result']
X_test = df_test.iloc[:,1:-1]


In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = X_train #sc.transform(X_train)
X_test_std = X_test #sc.transform(X_test)

In [9]:
from sklearn.linear_model import Perceptron
ppn = Perceptron(max_iter=40, eta0=0.1, random_state=0)
ppn.fit(X_train_std, y_train)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=0.1,
           fit_intercept=True, max_iter=40, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [10]:
y_pred = ppn.predict(X_test_std)
print('Misclassified samples: %d' % (y_test != y_pred).sum())

Misclassified samples: 2267


In [11]:
from sklearn.metrics import accuracy_score
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Accuracy: 0.53


In [12]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):
    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
    np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    # plot all samples
    X_test, y_test = X[test_idx, :], y[test_idx]
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],alpha=0.8, 
                    c=cmap(idx),marker=markers[idx], label=cl)
    # highlight test samples
    if test_idx:
        X_test, y_test = X[test_idx, :], y[test_idx]
        plt.scatter(X_test[:, 0], X_test[:, 1], 
                    c='',alpha=1.0, linewidth=1, 
                    marker='o',s=55, label='test set')

In [16]:
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))
plot_decision_regions(X=X_combined_std, y=y_combined, classifier=ppn,
                      test_idx=range(105,150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.show()

ValueError: X has 2 features per sample; expecting 12